### Bước 1: Tạo Park Session và import các thư viện cần thiết.


In [127]:
!pip install findspark
import findspark
findspark.init("/home/cuong/Downloads/spark-3.5.5-bin-hadoop3")

In [128]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/cuong/Downloads/spark-3.5.5-bin-hadoop3"

In [129]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,next_day,to_date, date_format, expr, lower, sum as spark_sum

spark = SparkSession.builder.appName("WeeklySKUQuantity").getOrCreate()


### Bước 2: Đọc file, xem các kiểu dữ liệu có phù hợp chưa, nếu chưa điều chỉnh lại cho đúng. 


In [130]:

df = spark.read.option("header", True).csv("file:///home/cuong/AmazonSaleReport.csv")
df.printSchema()


root
 |-- index: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Sales Channel : string (nullable = true)
 |-- ship-service-level: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Courier Status: string (nullable = true)
 |-- Qty: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- ship-city: string (nullable = true)
 |-- ship-state: string (nullable = true)
 |-- ship-postal-code: string (nullable = true)
 |-- ship-country: string (nullable = true)
 |-- promotion-ids: string (nullable = true)
 |-- B2B: string (nullable = true)
 |-- fulfilled-by: string (nullable = true)
 |-- Unnamed: 22: string (nullable = true)



In [131]:
df = df.withColumn("Qty", col("Qty").cast("int"))
df = df.withColumn("order_date", to_date(col('date'), "MM-dd-yy"))

### Bước 3: Sắp xếp các record theo thứ tự Date, SKU và filter các record có Status là Shipped.


In [132]:
df_sorted = df.orderBy("order_date", "SKU")
df_sorted = df_sorted.filter(lower(col("Status")).contains("shipped"))

### Bước 4: GroupBy 2 cột date và SKU, tính tổng cột Qty và lưu với tên total_quantity.


In [133]:
result_sorted = df_sorted.withColumn("report_date",next_day(col("order_date"),"Mon"))
result_sorted = result_sorted.groupBy(['report_date', 'SKU']).agg(spark_sum("Qty").alias("total_quantity"))

### Bước 5: Format lại dữ liệu theo yêu cầu và ghi ra file.


In [134]:

result_sorted = result_sorted.withColumn("report_date", date_format("report_date", "dd-MM-yyyy"))
result_sorted = result_sorted.withColumnRenamed("SKU", 'sku')
result_sorted.coalesce(1).write.option('header', True).csv("file:///home/cuong/res")